The purpose of this notebook is to demonstrate on a randomly selected diffusion weighted dataset how we can obtain a DTI fit, save it to a file, and save an FA image from it. The accompanying script `05.1_dti_fit.py` actually does this stuff to the full dataset. This notebook is just a demo to play with.

In [ ]:
import os
import glob
import random
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import dipy.io.image
import dipy.io
import dipy.core.gradients
import dipy.reconst.dti
import dipy.segment.mask

In [ ]:
data_dir = '/home/ebrahim/data/abcd/DMRI_extracted'
img_dirs = glob.glob(os.path.join(data_dir,'*ABCD-MPROC-DTI*/sub-*/ses-*/dwi/'))
output_dir = './dti_fit_images/'

In [ ]:
sampled_fmriresults01_df = pd.read_csv('01.0_abcd_sample/sampled_fmriresults01.csv')
sampled_fmriresults01_df['dirname'] = sampled_fmriresults01_df.derived_files.apply(lambda x : x.split('/')[-1].strip('.tgz'))
dirname_to_full_path = {img_dir.split('/')[-5]:img_dir for img_dir in img_dirs}

In [ ]:
data = []
for _,row in sampled_fmriresults01_df.iterrows():
    if row.dirname not in dirname_to_full_path.keys():
        raise FileNotFoundError(f"Could not find a directory for fmriresults01 id {row.fmriresults01_id}")
    img_dir = dirname_to_full_path[row.dirname]
    dwi_path = glob.glob(os.path.join(img_dir, '*.nii'))[0]
    bval_path = glob.glob(os.path.join(img_dir, '*.bval'))[0]
    bvec_path = glob.glob(os.path.join(img_dir, '*.bvec'))[0]
    data.append({
        'img_dir' : img_dir,
        'dwi_path' : dwi_path,
        'bval_path' : bval_path,
        'bvec_path' : bvec_path,
        'subjectkey' : row.subjectkey,
        'interview_age' : row.interview_age
    })

In [ ]:
# Run this to look at the number of bvals for each image, alongside subject id and interview age

for d in data:
    bvals, bvecs = dipy.io.read_bvals_bvecs(d['bval_path'], d['bvec_path'])
    print(f"{len(bvals)} \t {d['subjectkey']} \t {d['interview_age']}")

In [ ]:
d = random.choice(data)
img_data, affine = dipy.io.image.load_nifti(d['dwi_path'])
bvals, bvecs = dipy.io.read_bvals_bvecs(d['bval_path'], d['bvec_path'])
gtab = dipy.core.gradients.gradient_table(bvals, bvecs)

In [ ]:
img_data_masked, mask = dipy.segment.mask.median_otsu(img_data, vol_idx = range(img_data.shape[-1]))

In [ ]:
plt.imshow(mask[:,:,90].T, origin='lower')
plt.show()

In [ ]:
tensor_model = dipy.reconst.dti.TensorModel(gtab)

In [ ]:
tensor_fit = tensor_model.fit(img_data_masked)

In [ ]:
# If we wanted to obtain the FA image from the tensor_fit object, here is how to do that
fa = dipy.reconst.dti.fractional_anisotropy(tensor_fit.evals)

In [ ]:
np.isnan(fa).sum() # Notice there are no nans

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(20,10))
axs[0].imshow(fa[62,:,:].T, origin='lower', cmap='gray')
axs[1].imshow(fa[:,:,80].T, origin='lower', cmap='gray')
plt.show()

In [ ]:
# Save the lower triangular part, 
# i.e. the unique elements of the diffusion tensor in the order Dxx, Dxy, Dyy, Dxz, Dyz, Dzz
lt = tensor_fit.lower_triangular()
dipy.io.image.save_nifti('test_lt.nii.gz', lt, affine)

If you wanted to construct the FA image from the lower triangular part of the diffusion tensor, here's how:

In [ ]:
eig = dipy.reconst.dti.eig_from_lo_tri(lt) # has eigenvals and eigenvecs
eigvals = eig[:,:,:,:3] # take only the eigenvals
fa = dipy.reconst.dti.fractional_anisotropy(eigvals)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(20,10))
axs[0].imshow(fa[62,:,:].T, origin='lower', cmap='gray')
axs[1].imshow(fa[:,:,80].T, origin='lower', cmap='gray')
plt.show()